In [7]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [116]:
path = "../temp/*.nc"

def pre(ds):
    ds = ( ds.expand_dims( ESTADISTICA = int( 
        #ds["XLAT"].encoding["source"][-4:-3] )
        1 )
    
        ).rename_dims( { "XTIME":"MES" } 
        ).rename_vars( { "XLAT": "LATITUD", "XLONG": "LONGITUD",
        "XTIME": "MES", "Pcp": "PRECIPITACION" }
        ).drop_vars( ["XTIME_bnds"] )
        )
    return ds

ds = xr.open_mfdataset( path,
            combine = "nested", 
            concat_dim = "ESTADISTICA",
            parallel = True, preprocess = pre
            )

ds["MES"] = range(1, 13)
ds["ESTADISTICA"] = range(1, 5)
#ds = ds.swap_dims( ["south_north", "west_east", "MES", "ESTADISTICA"] )

ds.to_dataframe().reorder_levels( ["south_north",
    "west_east", "MES", "ESTADISTICA"] ).sort_index(
    ).to_xarray().set_coords( ["LONGITUD", "LATITUD"] )

<xarray.Dataset>
Dimensions:        (south_north: 112, west_east: 132, MES: 12, ESTADISTICA: 4)
Coordinates:
  * south_north    (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east      (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * MES            (MES) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * ESTADISTICA    (ESTADISTICA) int64 1 2 3 4
    LONGITUD       (south_north, west_east, MES, ESTADISTICA) float32 -100.5 ...
    LATITUD        (south_north, west_east, MES, ESTADISTICA) float32 18.14 ....
Data variables:
    PRECIPITACION  (south_north, west_east, MES, ESTADISTICA) float32 0.0 ......

In [65]:
est = ( [1, 2, 3, 4]
    * df.index.get_level_values("west_east").unique().shape[0]
    * df.index.get_level_values("south_north").unique().shape[0] ) * 12
len(est)

709632

In [80]:
ds["XLAT"].encoding["source"]

'/Users/rodrigo/Documents/Research/Code/cc_idt/results/prec_hist_hist_stats.nc'

In [79]:
path = "../results/prec_hist_hist_stats.nc"
ds = xr.open_dataset(path)
ds["XLAT"].encoding["source"][-4:-3]

's'

In [76]:
path = "../results/prec_hist_hist_stats.nc"
df = xr.open_dataset(path).to_dataframe().reset_index( 
    ["XTIME", "sfc", "bnds"] ).drop( ["XTIME_bnds", "bnds"], axis = 1
    ).rename( { "XLONG":"LONGITUD", "XLAT": "LATITUD", "XTIME": "MES",
    "Pcp":"PRECIPITACION", "sfc":"ESTADISTICAS" }, axis = 1 )
#df["ESTADISTICAS"] = est
df

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/rodrigo/Documents/Research/Code/cc_idt/temp/*.nc'

In [43]:
ds.rename( {"XLONG":"LONGITUD", "XLAT": "LATITUD", "XTIME": "MES",
    "Pcp":"PRECIPITACION", "sfc":"ESTADISTICAS"}).drop(["XTIME_bnds"])

<xarray.Dataset>
Dimensions:   (MES: 12, south_north: 112, west_east: 132)
Coordinates:
  * MES       (MES) datetime64[ns] 2015-01-16T11:30:00 ... 2015-12-16T11:30:00
    LONGITUD  (south_north, west_east) float32 -100.5 -100.4 ... -97.36 -97.33
    LATITUD   (south_north, west_east) float32 18.14 18.14 18.14 ... 20.63 20.63
Dimensions without coordinates: south_north, west_east
Data variables:
    *empty*
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    frequency:    mon
    history:      Tue Sep 05 21:50:18 2023: cdo -s merge temp/prec_hist_hist_...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [30]:
path = "../results/prec_hist_hist_tretorno.nc"
path_1 = "../results/prec_hist_hist_tretorno_1.nc"
df = xr.open_dataset(path).to_dataframe().reorder_levels(
    ["south_north", "west_east", "DURACION", "TIEMPO_RETORNO"] ).sort_index()
df_2 = df.copy().drop( ["PROBABILIDAD", "AÑO"], axis = 1 
    ).reset_index( "TIEMPO_RETORNO" )
#df_2 = df_2.reset_index( "DURACION" )
df_3 = df_2.copy().drop( ["INTENSIDAD", "TIEMPO_RETORNO"], axis = 1).groupby(
    ["south_north", "west_east", "DURACION"] ).mean()
cols = ["GEV_C", "GEV_LOC", "GEV_SCALE", "KTEST_P"]
df_3[ cols ] = None
df_3

LONGITUD    LATITUD GEV_C GEV_LOC GEV_SCALE  \
south_north west_east DURACION                                                  
0           0         1        -100.453476  18.135178  None    None      None   
                      2        -100.453476  18.135178  None    None      None   
                      3        -100.453476  18.135178  None    None      None   
                      4        -100.453476  18.135178  None    None      None   
                      5        -100.453476  18.135178  None    None      None   
...                                    ...        ...   ...     ...       ...   
111         131       72        -97.333252  20.628967  None    None      None   
                      84        -97.333252  20.628967  None    None      None   
                      96        -97.333252  20.628967  None    None      None   
                      108       -97.333252  20.628967  None    None      None   
                      120       -97.333252  20.628967  None    None      None   

                               KTEST_P  
south_north west_east DURACION          
0           0         1           None  
                      2           None  
                      3           None  
                      4           None  
                      5           None  
...                                ...  
111         131       72          None  
                      84          None  
                      96          None  
                      108         None  
                      120         None  

[325248 rows x 6 columns]

In [31]:
t = ( ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 )
    * df_2.index.get_level_values("west_east").unique().shape[0]
    * df_2.index.get_level_values("south_north").unique().shape[0]
    * df_2.index.get_level_values("DURACION").unique().shape[0] )

In [32]:
#d = [ 5/60, 10/60, 15/60, 20/60, 30/60, 40/60, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5,
#    6, 8, 10, 12, 14, 16, 18, 20, 22, 24 ] * 132 * 112 * 31
#t = ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 ) * 112 * 132 * 24
#df_2["DURACION"] = d
df_2["TIEMPO_RETORNO"] = t
df_2 = df_2.dropna()
#df_2 = df_2.set_index("DURACION", append = True).sort_values(
#    ["south_north", "west_east", "DURACION", "TIEMPO_RETORNO"] )
df_2

TIEMPO_RETORNO    LONGITUD    LATITUD  \
south_north west_east DURACION                                          
0           0         1                    5.0 -100.453476  18.135178   
                      1                   10.0 -100.453476  18.135178   
                      1                   25.0 -100.453476  18.135178   
                      1                   50.0 -100.453476  18.135178   
                      1                  100.0 -100.453476  18.135178   
...                                        ...         ...        ...   
111         131       120                 50.0  -97.333252  20.628967   
                      120                100.0  -97.333252  20.628967   
                      120                200.0  -97.333252  20.628967   
                      120                500.0  -97.333252  20.628967   
                      120               1000.0  -97.333252  20.628967   

                                INTENSIDAD  
south_north west_east DURACION              
0           0         1                0.0  
                      1                0.0  
                      1                0.0  
                      1                0.0  
                      1                0.0  
...                                    ...  
111         131       120              0.0  
                      120              0.0  
                      120              0.0  
                      120              0.0  
                      120              0.0  

[2601984 rows x 4 columns]

In [33]:
i = 90
j = 90
for k in df_3.index.get_level_values("DURACION").unique():
    params = stats.genextreme.fit(df.loc[ (i, j, k), "INTENSIDAD"] )
    pvalue = stats.kstest(df.loc[ (i, j, k), "INTENSIDAD"],
        stats.genextreme(*params).cdf ).pvalue
    df_3.loc[ (i, j, k), cols ] = [*params] + [pvalue]
    df_2.loc[ (i, j, k), "INTENSIDAD" ] = stats.genextreme(
        *df_3.loc[ (i, j, k), cols[:-1] ] ).isf(
        1 / df_2.loc[ (i, j, k), "TIEMPO_RETORNO" ].values )

In [34]:
df_2.loc[(90, 90, 4)]

TIEMPO_RETORNO   LONGITUD    LATITUD  \
south_north west_east DURACION                                         
90          90        4                    5.0 -98.309807  20.160126   
                      4                   10.0 -98.309807  20.160126   
                      4                   25.0 -98.309807  20.160126   
                      4                   50.0 -98.309807  20.160126   
                      4                  100.0 -98.309807  20.160126   
                      4                  200.0 -98.309807  20.160126   
                      4                  500.0 -98.309807  20.160126   
                      4                 1000.0 -98.309807  20.160126   

                                INTENSIDAD  
south_north west_east DURACION              
90          90        4          11.949419  
                      4          13.672395  
                      4          16.164227  
                      4          18.270647  
                      4          20.607351  
                      4          23.207680  
                      4          27.108141  
                      4          30.456661